In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [108]:
player_df = pd.read_csv('../data/new/players_all.csv')
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4742 entries, 0 to 4741
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        4742 non-null   int64 
 1   No.       4742 non-null   int64 
 2   Name      4742 non-null   object
 3   Age       4742 non-null   int64 
 4   Team      4742 non-null   object
 5   Position  4742 non-null   object
 6   year      4742 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 259.5+ KB


In [109]:
stats1_df = pd.read_csv('../data/new/1xbet_all.csv')
stats1_df = stats1_df.drop(['Team','Age','Position'], axis=1)
stats1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4756 entries, 0 to 4755
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      4756 non-null   object 
 1   Apps      4756 non-null   int64  
 2   Mins      4756 non-null   int64  
 3   Goals     4756 non-null   int64  
 4   Assists   4756 non-null   int64  
 5   SpG       4756 non-null   float64
 6   KeyP      4756 non-null   float64
 7   Drb_x     4756 non-null   float64
 8   Fouled    4756 non-null   float64
 9   Off       4756 non-null   float64
 10  Disp      4756 non-null   float64
 11  UnsTch    4756 non-null   float64
 12  Rating    4756 non-null   float64
 13  Tackles   4756 non-null   float64
 14  Inter     4756 non-null   float64
 15  Fouls     4756 non-null   float64
 16  Offsides  4756 non-null   float64
 17  Clear     4756 non-null   float64
 18  Drb_y     4756 non-null   float64
 19  Blocks    4756 non-null   float64
 20  year      4756 non-null   int6

In [110]:
stats2_df = pd.read_csv('../data/new/understat_all.csv')
stats2_df = stats2_df.drop(['No','Team'], axis=1)
stats2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4755 entries, 0 to 4754
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       4755 non-null   object 
 1   Apps         4755 non-null   int64  
 2   Min          4755 non-null   int64  
 3   G            4755 non-null   int64  
 4   NPG          4755 non-null   int64  
 5   A            4755 non-null   int64  
 6   xG           4755 non-null   float64
 7   NPxG         4755 non-null   float64
 8   xA           4755 non-null   float64
 9   xGChain      4755 non-null   float64
 10  xGBuildup    4755 non-null   float64
 11  xG90         4755 non-null   float64
 12  NPxG90       4755 non-null   float64
 13  xA90         4755 non-null   float64
 14  xG90+xA90    4755 non-null   float64
 15  NPxG90+xA90  4755 non-null   float64
 16  xGChain90    4755 non-null   float64
 17  xGBuildup90  4755 non-null   float64
 18  year         4755 non-null   int64  
dtypes: flo

In [111]:
salary_df = pd.read_csv('../data/new/salary_all0.csv')
salary_df = salary_df.drop(['Position', 'Team'], axis=1)
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           4803 non-null   float64
 1   Name           4803 non-null   object 
 2   Weekly Salary  4803 non-null   int64  
 3   year           4803 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 150.2+ KB


In [112]:
# player_df, salary_df, stats1_df, stats2_df
eda_df = player_df.merge(salary_df[['Weekly Salary','Name', 'year']], how='left', left_on=['Name', 'year'], right_on=['Name', 'year'])
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4763 entries, 0 to 4762
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             4763 non-null   int64  
 1   No.            4763 non-null   int64  
 2   Name           4763 non-null   object 
 3   Age            4763 non-null   int64  
 4   Team           4763 non-null   object 
 5   Position       4763 non-null   object 
 6   year           4763 non-null   int64  
 7   Weekly Salary  3257 non-null   float64
dtypes: float64(1), int64(4), object(3)
memory usage: 297.8+ KB


In [113]:
eda_df.isnull().sum(axis=0)

Id                  0
No.                 0
Name                0
Age                 0
Team                0
Position            0
year                0
Weekly Salary    1506
dtype: int64

In [114]:
eda_df.isnull().mean()

Id               0.000000
No.              0.000000
Name             0.000000
Age              0.000000
Team             0.000000
Position         0.000000
year             0.000000
Weekly Salary    0.316187
dtype: float64

In [115]:
na_list = eda_df.loc[eda_df.isnull().any(axis=1)].loc[:, ['Name','year']]
na_list.groupby('year')['Name'].count()

year
2014    378
2015    306
2016    156
2017    101
2018     80
2019    105
2020    112
2021    122
2022    146
Name: Name, dtype: int64

In [116]:
player_df.groupby('year')['Name'].count()

year
2014    535
2015    547
2016    529
2017    516
2018    497
2019    511
2020    519
2021    533
2022    555
Name: Name, dtype: int64

In [117]:
eda_df.groupby('year')['Name'].count()

year
2014    538
2015    550
2016    532
2017    518
2018    499
2019    514
2020    521
2021    536
2022    555
Name: Name, dtype: int64

In [118]:
eda_df[eda_df['Weekly Salary'].isnull()].head()

,Id,No.,Name,Age,Team,Position,year,Weekly Salary
5,5,1185,Mile Jedinak,30,Crystal Palace,Midfielder,2014,NaN
6,6,1333,Phil Jones,22,Man Utd,Defender,2014,NaN
7,7,1201,Morgan Schneiderlin,24,Southampton,Midfielder,2014,NaN
12,12,1391,Robert Huth,30,Leicester,Defender,2014,NaN
14,14,708,James Tomkins,25,West Ham,Defender,2014,NaN


In [119]:
salary_df.groupby('year')['Name'].count()

year
2014    226
2015    374
2016    577
2017    588
2018    587
2019    648
2020    631
2021    609
2022    563
Name: Name, dtype: int64

In [120]:
# player_df, salary_df, stats1_df, stats2_df
eda_df = player_df.merge(stats2_df, how='outer', left_on=['Name', 'year'], right_on=['Player', 'year'])
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5037 entries, 0 to 5036
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           4744 non-null   float64
 1   No.          4744 non-null   float64
 2   Name         4744 non-null   object 
 3   Age          4744 non-null   float64
 4   Team         4744 non-null   object 
 5   Position     4744 non-null   object 
 6   year         5037 non-null   int64  
 7   Player       4837 non-null   object 
 8   Apps         4837 non-null   float64
 9   Min          4837 non-null   float64
 10  G            4837 non-null   float64
 11  NPG          4837 non-null   float64
 12  A            4837 non-null   float64
 13  xG           4837 non-null   float64
 14  NPxG         4837 non-null   float64
 15  xA           4837 non-null   float64
 16  xGChain      4837 non-null   float64
 17  xGBuildup    4837 non-null   float64
 18  xG90         4837 non-null   float64
 19  NPxG90

In [121]:
na_df = eda_df[eda_df.isnull().any(axis=1)]
na1 = na_df[na_df['Apps'].isnull()]
na1[['Name', 'year']].to_excel('../data/dummy/na1.xlsx')

PermissionError: [Errno 13] Permission denied: '../data/dummy/na1.xlsx'

In [ ]:
na2 = na_df[~na_df['Apps'].isnull()]
na2[['Player','year']].to_excel('../data/dummy/na2.xlsx')